In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import ast
# import evaluate
import numpy as np
from sklearn.metrics import f1_score

In [2]:
def add_ast(example):
    example["ner_tags"] = ast.literal_eval(example["ner_tags"])
    example["tokens"] = ast.literal_eval(example["tokens"])
    return example

In [3]:
dataset = load_dataset("csv", data_files="data/train.csv")["train"]
dataset = dataset.remove_columns("processed_text")
dataset = dataset.remove_columns("target_labels_positions")
dataset = dataset.remove_columns("label")
dataset = dataset.remove_columns("strat")
dataset = dataset.train_test_split(test_size=0.15, seed=42)
dataset = dataset.rename_column("label_ids", "ner_tags")
dataset = dataset.rename_column("processed_text_split", "tokens")
dataset = dataset.map(add_ast)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 423
    })
    test: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 75
    })
})

In [5]:
model_name = "ai-forever/sbert_large_mt_nlu_ru"
output_dir = "baseline_ai-forever-sbert_large_mt_nlu_ru_v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        # Map tokens to their respective word.
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            # Only label the first token of a given word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [8]:
label2id = {
    "O": 0,
    "B-discount": 1,
    "B-value": 2,
    "I-value": 3
}
id2label = {
    0: "O",
    1: "B-discount",
    2: "B-value",
    3: "I-value"
}

In [9]:
from sklearn.metrics import precision_recall_fscore_support


def weighted_f1(y_true, y_pred, weights):
    """Расчет взвешенной F1 меры с индивидуальными весами для классов."""

    y_true_flat = []
    y_pred_flat = []

    for prediction, label in zip(y_pred, y_true):
        for pr, le in zip(prediction, label):
            if le == -100:
                continue
            else:
                y_true_flat.append(le)
                y_pred_flat.append(pr)

    # рассчитываем F1 для всех классов
    _, _, f1, support = precision_recall_fscore_support(y_true_flat, y_pred_flat, average=None,
                                                        labels=list(weights.keys()))

    # вычисление взвешенной F-меры
    weighted_f1 = np.sum(f1 * [weights[label] for label in np.unique(y_true_flat + y_pred_flat)] * support) / np.sum(
        support * [weights[label] for label in np.unique(y_true_flat + y_pred_flat)])
    return weighted_f1


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    class_weights = {
        1: 1,
        2: 2,
        3: 2,
        0: 0
    }

    weighted_f1_score = weighted_f1(labels, predictions, class_weights)

    return {
        "f1": weighted_f1_score,
    }

In [10]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/sbert_large_mt_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/530 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.019432175904512405, 'eval_f1': 0.9937436839649448, 'eval_runtime': 1.6049, 'eval_samples_per_second': 46.731, 'eval_steps_per_second': 6.231, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.01454983837902546, 'eval_f1': 0.9955436337784025, 'eval_runtime': 1.4435, 'eval_samples_per_second': 51.958, 'eval_steps_per_second': 6.928, 'epoch': 2.0}


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.019655821844935417, 'eval_f1': 0.9965762074408555, 'eval_runtime': 1.6197, 'eval_samples_per_second': 46.305, 'eval_steps_per_second': 6.174, 'epoch': 9.0}
{'loss': 0.0134, 'grad_norm': 0.11675046384334564, 'learning_rate': 1.1320754716981133e-06, 'epoch': 9.43}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.01983506791293621, 'eval_f1': 0.9968583886315955, 'eval_runtime': 1.642, 'eval_samples_per_second': 45.676, 'eval_steps_per_second': 6.09, 'epoch': 10.0}
{'train_runtime': 351.0664, 'train_samples_per_second': 12.049, 'train_steps_per_second': 1.51, 'train_loss': 0.012717333991291388, 'epoch': 10.0}


TrainOutput(global_step=530, training_loss=0.012717333991291388, metrics={'train_runtime': 351.0664, 'train_samples_per_second': 12.049, 'train_steps_per_second': 1.51, 'total_flos': 3816559961389224.0, 'train_loss': 0.012717333991291388, 'epoch': 10.0})

In [12]:
import os

trainer.save_model(os.path.join(output_dir, "final"))